# Parameterized queries with SQL and Python

This notebook covers:
1) what are SQL parameterized queries
2) what is the SQL injection risk
3) a python best practice to avoid SQL injection risk

## Pre-requisites

- Have PostgreSQL set up on your machine 
- Have psycopg2 package installed (PG SQL database adaptater for python)

In [ ]:
import os
import configparser
import psycopg2

## Intro - Creating a DB, a table and a few records

1) Connect to a template DB
2) Create the DB, if it doesn't exist already
3) Close the connection to template DB
4) Connect to created DB
5) Create a users table with username and password
6) Insert a few records

In [ ]:
# calling config file
config = os.environ.get("CONFIG")
cfg = configparser.ConfigParser()
cfg.read(config)

# retrieving DB details
dbname = cfg.get("TEMPLATE_DB", "database")
user = cfg.get("TEMPLATE_DB", "username")
password = cfg.get("TEMPLATE_DB", "password")
host = cfg.get("TEMPLATE_DB", "host")

# Connect to a template DB prior to creating a demo DB
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host
)

conn.autocommit = True

cursor = conn.cursor()

In [ ]:
# creating a DB for SQL injection purposes
create_db_query = """
CREATE DATABASE sql_injection_db
"""

cursor.execute(create_db_query)

In [ ]:
# closing connection
conn.close()

In [ ]:
# retrieving DB details
dbname = cfg.get("SQL_INJECTION_DB", "database")
user = cfg.get("SQL_INJECTION_DB", "username")
password = cfg.get("SQL_INJECTION_DB", "password")
host = cfg.get("SQL_INJECTION_DB", "host")

# Connect to the SQL injection DB
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host
)

conn.autocommit = True

cursor = conn.cursor()

In [ ]:
# creating a users table with 3 columns (ID, username, password)
# note: it is good practice to set a primary key (PK) and some rules (e.g. unique, not null) when creating tables
create_table_and_insert_records_query = """
    CREATE TABLE IF NOT EXISTS users (
        id SERIAL PRIMARY KEY,
        username VARCHAR(50) UNIQUE NOT NULL,
        password VARCHAR(50) NOT NULL
    );
    INSERT INTO users (username, password) VALUES
    ('user1', 'password1'),
    ('user2', 'password2'),
    ('user3', 'password3'),
    ('user4', 'password4'),
    ('user5', 'password5'),
    ('user6', 'password6');
"""

cursor.execute(create_table_and_insert_records_query)

## 1 - Parameterized query: an easy way to deal with user inputs and a database

There are plenty of situations where you would likely want to send the same query, but with one or more variables to adapt the query a bit.

Say you have a web interface where a user can create its credentials: a username and a password are required as inputs.
- The query is always the same
- The inputs (username, password) change

In this case, a parameterized query, taking input values as parameters, is useful!

In [ ]:
# using username and password as variables
username = "variable_user_name"
password = "variable_password"

# using a formatted string to pass these variables
create_account_parameterized_query = """
    INSERT INTO users (username, password) VALUES
    ('{0}', '{1}');
""".format(username, password)

cursor.execute(create_account_parameterized_query)

## 2 - SQL injection risk

The problem with a parameterized query is the parameter / variable being passed.
<br>The parameter can be anything and a malicious user could send a dangerous query to a database.
<br><strong>Risks related to SQL injection can be theft, alteration or even destruction of sensitive data.</strong>

<br>In the example below, the user injects a SQL query through the password variable and retrieves all the stored credentials from the DB

In [ ]:
# using username and password as variables
username = "malicious_user"
password = "fake_password\');SELECT * FROM users--"

# using a formatted string to pass these variables
sql_injection_query = """
    INSERT INTO users (username, password) VALUES
    ('{0}', '{1}');
""".format(username, password)

cursor.execute(sql_injection_query)

# records cannot be stolen this time
try: 
    stolen_records = cursor.fetchall()
    print("Successful SQL injection")
except: 
    print("SQL injection attempt failed")

## 3 - SQL injection analysis prevention

How did the above example work?
- After the input password, the semicolon ";" indicates a new query
- At the end of the query, the "--" aims at transforming whatever comes after the placeholder into comments
- The parameter can be anything and a malicious user could send a dangerous query to a database.

How to prevent from SQL injection risks?
- Pass the variables separately, as parameters, instead of dynamically constructing the SQL query through concatenation / string formatting
- Check package documentation: packages like psycopg2 will provide the necessary information to prevent SQL injection (https://www.psycopg.org/psycopg3/docs/basic/params.html)
- Check for more adavanced security measures online: while the above is going to significantly reduce the risk of SQL injections, it is not failproof

In the example below, the difference is that placeholder strings (%s) are used, leveraging the package's capabilities.
<br>Here, the variable is interpreted as a value and not as arbitrary SQL code, preventing from SQL injections.

In [ ]:
# using username and password as variables
username = "malicious_user2"
password = "fake_password\');SELECT * FROM users--"

# using a formatted string to pass these variables
sql_injection_query = """
    INSERT INTO users (username, password) VALUES
    (%s, %s);
"""

# variables are passed as tuple, using the package's capabilities
cursor.execute(sql_injection_query, (username, password))

# records cannot be stolen this time
try: 
    stolen_records = cursor.fetchall()
    print("Successful SQL injection")
except: 
    print("SQL injection attempt failed")

## Wrap up

Optional: 
- deleting the table created
- deleting the DB created
- closing connections

In [ ]:
# closing connection to sql_inection database
conn.close()

In [ ]:
# retrieving DB details
dbname = cfg.get("TEMPLATE_DB", "database")
user = cfg.get("TEMPLATE_DB", "username")
password = cfg.get("TEMPLATE_DB", "password")
host = cfg.get("TEMPLATE_DB", "host")

# Connect to a template DB prior to creating a demo DB
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host
)

conn.autocommit = True

cursor = conn.cursor()

In [ ]:
# deleting the sql_injection database
remove_db = """DROP DATABASE sql_injection_db"""

cursor.execute(remove_db)

# closing connection to template DB
conn.close()